In [1]:
# Notebook'u yayınladığım site bu işi bilim adına gönüllü olarak yapıyor. 

# Henüz hesap ve dosya erişim sistemini oluşturmadılar.

# Diğerlerininde faydalanmasını sağlamak için lütfen notebook'u 

# Sadece Çalıştırın Değiştirmeyin

# Düzenlemek İçin File -> Make a Copy yolunu kullanarak notebook'u klonlayabilirsiniz


    Tensorflow C++ da yazıdı bu yüzden pythondaki verileri doğrudan kütüphanede kullanamıyoruz.
    Verileri kullanabilmek için tensorflowun bize sunduğu veri yapılarını kullanmamız gerekiyor.

    - Variable() değişkenleri
    - constant() sabit değerleri
    - placeholder() geçici olarak değer tutan veri yapılarını temsil ediyor

    Bu veri yapılarında çeşitli boyutlardaki matrisleri tutabiliriz. 
    Veri türü olarak python listelerini, numpy dizilerini veya tensorflow dizilerini kullanabiliriz.
    Ayrıca istersek tanımladığımız veri yapılarını adlandırabiliriz. Tensorflow öntanımlı olarak 32 bit float
    veriler ile işlemler yapıyor.

    Kullanımı:
        v = tf.Variable(matris, name="isim")
        p = tf.placeholder(matris, name="isim")
        c = tf.constant(matris, name="isim")



    Yapay Sinir Hucresi:
$$Sinyal = AktivasyonFonksiyonu({AgirlikMatrisi * Girdi + SapmaDegeri})$$

    Agirlik matrisleri (weights) yapay sinir aglarindaki hafizayi temsil ediyor.
    Yapay Sinir Aginda bu matrisleri, gelen veriler ve sonrasinda kullandigimiz optimizasyon algoritmasi
    ile sekillendiriyoruz. Yapay sinir agi boyle ogreniyor.

  
    
    

      Agirlik matrisini sadece girdilerle sekillendirerek aktivasyon fonksiyonunu esnek bir sekilde kullanmayiz. 
      Aktivasyon fonksiyonlarından biri olan sigmoid fonksiyonunun biassız ve bias eklenmiş grafiklerini
      inceleyelim.
$$ sigmoid: y = 1/(1+e^{-x}) $$

![](neuron_1.gif)![](sigmoid_1.png) ![](neuron_2.gif) ![](sigmoid_2.png)
    
    Aktivasyon fonksiyonu yapay sinir hucresinin urettigi sinyalin gucune gore hucrenin aktif olup olmayacagini
    veya ne olcude aktif olacagini belirler.
    sigmoid, tanh, step bu fonksiyonlara örnek verilebilir 

    Yapay zekanın ogrenmesini saglamak icin tahmin edilen veriler ile dogru verileri kıyaslayıp,
    aralarındaki kaybi(hatayi) hesaplayip azaltmamiz gerekiyor
    
    Bu hatayi hesaplamak icin;
        loss = tf.reduce_mean(tf.square(y_pred - y_real))    

    islemini kullanıyoruz. Burada oncelikle iki deger arasindaki farkı hesapladık. 
    Ardindan belirginlestirmek icin farkin karesini aldik. Son olarak da reduce_mean() 
    fonksiyonu ile kaybin ortalama degerini olctuk. Bu fonksiyon parametre olarak 
    input_tensor (giris matrisini) aliyor. Ek olarak hangi boyuta gore ortalama alinacagi
    belirlenebiliyor.

    Bu hatayi optimizasyon algoritmalari ile azaltabiliriz. Tensorflow bize hazir kullanabilecegimiz 
    bazi optimizasyon algoritmalari sunuyor. Biz bunlar arasindan GradientDescentOptimizer'i kullanacagiz.
![](gdo.png)

    Bu algoritma N boyutlu hata matrisinde minumum noktayi bulmayi hedefliyor.
    Bu algoritmayi;
        optimizer = tf.train.GradientDescentOptimizer(0.5)
    seklinde kullanabiliriz. Girdigimiz 0.5 degeri ogrenme oranini temsil ediyor. 
    Bu degeri cok yuksek yaparsak minumum noktayi es gecebiliriz, cok kucuk yaparsak da
    minimum noktaya ulasmamiz cok uzun surebilir. 
    
    Tensorflow "lazy" calisiyor yani tanimladigimiz degiskenler, fonsiyonlar, optimizasyon algoritmalari 
    tanimladigimiz anda calistirilmiyor. Bu islemleri aktif etmek icin tensorflow oturumu (Session) kullanmamiz
    gerekiyor. Session'u bir isaretci olarak dusunulebilir. Tanimladigimiz yapay sinir agi modelinin istedigimiz
    adimini Session ile calistirip degerini alabiliriz.

    Asagida en basit haliyle bir yapay sinir agi olusturduk. Giris verileri icin (X) -1 ile 1 arasinda rastgele sayilardan olusan 3x3 bir matris olusturduk.
    Gercek verileri temsil etmek icinde giris verilerinde biraz oynama yaptik. Bu ornekteki amacimiz giris matrisini(X) ile yapay sinir agini egiterek her adimda
    gercek degerlere(y_real) biraz daha yaklasmak. 


In [2]:
import numpy as np
import tensorflow as tf

In [3]:
# Rastgele veriler oluşturduk
X = np.random.rand(3, 3).astype(np.float32)
y_real = X * 0.1 + 0.3
print(y_real)

[[ 0.30501819  0.37679443  0.39487767]
 [ 0.32714039  0.36743698  0.32770458]
 [ 0.38853815  0.38401783  0.38375115]]


In [4]:
weights = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) # 1D array,  [-0.322323, 0.612312, ... , 0.712342]
biases = tf.Variable(tf.zeros([1])) # [0]

In [5]:
y_pred = weights * X + biases # giris verilerine gore bir tahmin olustur (sinir hucresi)

In [6]:
loss = tf.reduce_mean(tf.square(y_pred - y_real)) # kaybi hesapla

In [7]:
optimizer = tf.train.GradientDescentOptimizer(0.5) # learning_rate < 1
train = optimizer.minimize(loss) # modeli optimize et

    Eğer Variable() tanımlamışsak tensorflowun  bu değişkenleri tanıması için 
    tf.global_variables_initializer() fonksiyonunu çalıştırmamız gerekiyor.

In [8]:
init = tf.global_variables_initializer() # important, degiskenleri tanimla

    Eğitimin her aşamasında modelin tahmit ettiği değerlerin asıl değerlere yaklaşmasını umuyoruz

In [9]:
# Train
sess = tf.Session() # pointer
sess.run(init) # degiskenler tanimlandi
for step in range(1000):
    sess.run(train) # Modeli 1 kez egittik. train -> optimizer -> loss -> (y_pred, y_real) -> Wx_plus_B -> W, X, B

    if step % 200 is 0: # 200 adimda bir
        # agirlik matrisini ve sapma degerini yazdirdik. (Amacimiz  y_pred'in, y_real'e yaklasmasi)
        print(step, sess.run(weights), sess.run(biases)) 

0 [ 0.02979365] [ 0.60090828]
200 [ 0.09999987] [ 0.3000001]
400 [ 0.09999988] [ 0.3000001]
600 [ 0.09999988] [ 0.3000001]
800 [ 0.09999988] [ 0.3000001]


In [10]:
prediction = sess.run(y_pred) # Modelden verileri almak için run metodunu bir değişkene atıyoruz
print(prediction, end="\n\n")
print(y_real)

[[ 0.30501825  0.3767944   0.39487764]
 [ 0.32714045  0.36743701  0.32770464]
 [ 0.38853812  0.38401783  0.38375115]]

[[ 0.30501819  0.37679443  0.39487767]
 [ 0.32714039  0.36743698  0.32770458]
 [ 0.38853815  0.38401783  0.38375115]]
